In [10]:

import sys
sys.path.append('./tf_pipeline/')

from tf_pipeline.tf_utils import create_mlp_softmax

from tf_pipeline.conf import *
import pandas as pd
import os 
import numpy as np 
from datetime import datetime 


def create_dt(horizon="validation", tr_last=1913):
    prices = pd.read_csv(os.path.join(RAW_PATH, "sell_prices.csv"), dtype=PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()

    cal = pd.read_csv(os.path.join(RAW_PATH, "calendar.csv"), dtype=CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()

    numcols = [f"d_{day}" for day in range(1, tr_last + 1)]
    catcols = ["id", "item_id", "dept_id", "store_id", "cat_id", "state_id"]
    dtype = {numcol: "float32" for numcol in numcols}
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv(
        os.path.join(RAW_PATH, "sales_train_%s.csv" % horizon),
        usecols=catcols + numcols,
        dtype=dtype,
    )

    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()

    increasing_term = dt.groupby(["dept_id", "store_id"])[numcols].sum()
    increasing_term = (
                              increasing_term.T - increasing_term.T.shift(28)
                      ) / increasing_term.T.shift(28)
    increasing_term = increasing_term.reset_index(drop=True).iloc[-365:, :]
    rates = increasing_term[increasing_term.abs() < 1].mean() + 1
    rates = rates.reset_index().rename(columns={0: "rate"})

    for day in range(tr_last + 1, tr_last + 2 * 28 + 1):
        dt[f"d_{day}"] = np.nan

    dt = pd.melt(
        dt,
        id_vars=catcols,
        value_vars=[col for col in dt.columns if col.startswith("d_")],
        var_name="d",
        value_name="sales",
    )

    dt = dt.merge(cal, on="d", copy=False)
    dt = dt.merge(prices, on=["store_id", "item_id", "wm_yr_wk"], copy=False)
    dt = dt.merge(rates, how="left")

    return dt

def compute_share(dt):
    shares = (
        dt.groupby(["dept_id", "store_id", "date"])["sales"]
        .sum()
        .reset_index()
        .rename(columns={"sales": "gp_sales"})
    )
    dt = dt.merge(shares, how="left")
    dt["sales"] = dt["sales"] / dt["gp_sales"]
    dt.drop(["gp_sales"], axis=1, inplace=True)
    return dt

def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id", "sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins:
        for lag, lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = (
                dt[["id", lag_col]]
                    .groupby("id")[lag_col]
                    .transform(lambda x: x.rolling(win).mean())
            )

    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
    }

    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")
    return dt

def train_and_pred(horizon="validation"):

    if horizon=="validation":
        tr_last = 1913
        fday = datetime(2016, 4, 25)
    elif horizon=="evaluation":
        tr_last = 1941
        fday = datetime(2016, 4, 25) + timedelta(days=28)
    else:
        raise ValueError('Wrong horizon arg.')

    dataframe = create_dt(horizon, tr_last)
    dataframe = compute_share(dataframe)
    dataframe = create_fea(dataframe)
    dataframe.dropna(inplace=True)
    

    list_preds = list()

    for _, df_gp in dataframe.groupby(['store_id', 'dept_id']):

        cat_feats = ['wday', 'quarter']

        n_items = len(df_gp['item_id'].drop_duplicates())    

        ids = df_gp[['id', 'item_id']].drop_duplicates()\
                                      .sort_values('item_id')['id']\
                                      .tolist()
        X = df_gp[
            ['d', 'item_id', 'wday', 'quarter', 'date', 'rmean_28_28', 'sales']
        ].pivot_table(index=['d', 'date', 'wday', 'quarter'],
                      columns=['item_id'],
                      values=['rmean_28_28', 'sales']).fillna(0)

        num_feats = ['_'.join(list(map(str, c))) for c in X.columns.tolist()]
        X.columns = num_feats

        target_feats = num_feats[n_items:]
        num_feats = num_feats[:n_items]
        X = X.reset_index()

        X_train = X[X['date']<fday][num_feats+cat_feats]
        X_test = X[X['date']>=fday][num_feats+cat_feats]

        input_dict_train = {'input_%s' % c: X_train[c] for c in num_feats+cat_feats}
        input_dict_test = {'input_%s' % c: X_test[c] for c in num_feats+cat_feats}

        cardinality = X[cat_feats].nunique()

        y_train = X[X['date']<fday][target_feats].values

        mlp = create_mlp_softmax(layers_list=[2048, 2048],
                                 output_count=n_items,
                                 cat_feats=cat_feats,
                                 cardinality=cardinality,
                                 num_feats=num_feats)



        training_params = {
                    'x': input_dict_train,
                    'y': y_train,
                    'batch_size': 128,
                    'epochs': 20,
                    'shuffle': True,
                }

        mlp.fit(**training_params)
        preds = mlp.predict(input_dict_test)
        preds = pd.DataFrame(preds,
                             index=['F%s' % c for c in range(1,29)],
                             columns=ids).T
        list_preds.append(preds)

    preds = pd.concat(list_preds)
    preds = preds.reset_index()
    preds.columns = ['id'] + preds.columns.tolist()[1:]
    
    preds.to_csv(
            os.path.join(EXTERNAL_PATH, "tf_weights_%s.csv" % horizon), index=False
        )

In [ ]:
horizon="evaluation"

if horizon=="validation":
    tr_last = 1913
    fday = datetime(2016, 4, 25)
elif horizon=="evaluation":
    tr_last = 1941
    fday = datetime(2016, 4, 25) + timedelta(days=28)
else:
    raise ValueError('Wrong horizon arg.')

dataframe = create_dt(horizon, tr_last)
dataframe = compute_share(dataframe)
dataframe = create_fea(dataframe)

In [12]:
list_preds = list()

for _, df_gp in dataframe.groupby(['store_id', 'dept_id']):
    
    cat_feats = ['wday', 'quarter']

    n_items = len(df_gp['item_id'].drop_duplicates())    
    
    ids = df_gp[['id', 'item_id']].drop_duplicates()\
                                  .sort_values('item_id')['id']\
                                  .tolist()
    X = df_gp[
        ['d', 'item_id', 'wday', 'quarter', 'date', 'rmean_28_28', 'sales']
    ].pivot_table(index=['d', 'date', 'wday', 'quarter'],
                  columns=['item_id'],
                  values=['rmean_28_28', 'sales']).fillna(0)

    num_feats = ['_'.join(list(map(str, c))) for c in X.columns.tolist()]
    X.columns = num_feats
    
    target_feats = num_feats[n_items:]
    num_feats = num_feats[:n_items]
    X = X.reset_index()

    X_train = X[(X['date']<fday) & (X['date']>=fday - timedelta(days=364))][num_feats+cat_feats]
    X_test = X[X['date']>=fday][num_feats+cat_feats]
    
    input_dict_train = {'input_%s' % c: X_train[c] for c in num_feats+cat_feats}
    input_dict_test = {'input_%s' % c: X_test[c] for c in num_feats+cat_feats}
    
    cardinality = X[cat_feats].nunique()

    y_train = X[(X['date']<fday) & (X['date']>=fday - timedelta(days=364))][target_feats].values
    
    mlp = create_mlp_softmax(layers_list=[2048, 2048],
                             output_count=n_items,
                             cat_feats=cat_feats,
                             cardinality=cardinality,
                             num_feats=num_feats)
    
    
   
    training_params = {
                'x': input_dict_train,
                'y': y_train,
                'batch_size': 128,
                'epochs': 20,
                'shuffle': True,
            }

    mlp.fit(**training_params)
    preds = mlp.predict(input_dict_test)
    preds = pd.DataFrame(preds,
                         index=['F%s' % c for c in range(1,29)],
                         columns=ids).T
    list_preds.append(preds)

preds = pd.concat(list_preds)

Epoch 1/20
3/3 [==============================] - 0s 43ms/step - loss: 6.9546 - root_mean_squared_error: 0.0168
Epoch 2/20
3/3 [==============================] - 0s 37ms/step - loss: 7.5331 - root_mean_squared_error: 0.0223
Epoch 3/20
3/3 [==============================] - 0s 39ms/step - loss: 6.7776 - root_mean_squared_error: 0.0122
Epoch 4/20
3/3 [==============================] - 0s 42ms/step - loss: 6.6575 - root_mean_squared_error: 0.0101
Epoch 5/20
3/3 [==============================] - 0s 40ms/step - loss: 6.5204 - root_mean_squared_error: 0.0090
Epoch 6/20
3/3 [==============================] - 0s 44ms/step - loss: 6.3412 - root_mean_squared_error: 0.0095
Epoch 7/20
3/3 [==============================] - 0s 46ms/step - loss: 6.1849 - root_mean_squared_error: 0.0080
Epoch 8/20
3/3 [==============================] - 0s 71ms/step - loss: 5.9745 - root_mean_squared_error: 0.0073
Epoch 9/20
3/3 [==============================] - 0s 39ms/step - loss: 5.8073 - root_mean_squared_error:

3/3 [==============================] - 0s 42ms/step - loss: 5.8859 - root_mean_squared_error: 0.0041
Epoch 14/20
3/3 [==============================] - 0s 43ms/step - loss: 5.8793 - root_mean_squared_error: 0.0042
Epoch 15/20
3/3 [==============================] - 0s 43ms/step - loss: 5.8426 - root_mean_squared_error: 0.0041
Epoch 16/20
3/3 [==============================] - 0s 44ms/step - loss: 5.8256 - root_mean_squared_error: 0.0039
Epoch 17/20
3/3 [==============================] - 0s 45ms/step - loss: 5.8222 - root_mean_squared_error: 0.0040
Epoch 18/20
3/3 [==============================] - 0s 41ms/step - loss: 5.8085 - root_mean_squared_error: 0.0041
Epoch 19/20
3/3 [==============================] - 0s 39ms/step - loss: 5.7920 - root_mean_squared_error: 0.0038
Epoch 20/20
3/3 [==============================] - 0s 43ms/step - loss: 5.7861 - root_mean_squared_error: 0.0039
Epoch 1/20
3/3 [==============================] - 0s 31ms/step - loss: 6.9364 - root_mean_squared_error: 0.0

3/3 [==============================] - 0s 41ms/step - loss: 6.2747 - root_mean_squared_error: 0.0086
Epoch 6/20
3/3 [==============================] - 0s 40ms/step - loss: 6.1389 - root_mean_squared_error: 0.0081
Epoch 7/20
3/3 [==============================] - 0s 42ms/step - loss: 5.9868 - root_mean_squared_error: 0.0074
Epoch 8/20
3/3 [==============================] - 0s 41ms/step - loss: 5.8891 - root_mean_squared_error: 0.0070
Epoch 9/20
3/3 [==============================] - 0s 39ms/step - loss: 5.7775 - root_mean_squared_error: 0.0068
Epoch 10/20
3/3 [==============================] - 0s 38ms/step - loss: 5.6910 - root_mean_squared_error: 0.0066
Epoch 11/20
3/3 [==============================] - 0s 38ms/step - loss: 5.6198 - root_mean_squared_error: 0.0065
Epoch 12/20
3/3 [==============================] - 0s 41ms/step - loss: 5.5679 - root_mean_squared_error: 0.0064
Epoch 13/20
3/3 [==============================] - 0s 41ms/step - loss: 5.5164 - root_mean_squared_error: 0.0062

3/3 [==============================] - 0s 45ms/step - loss: 5.8334 - root_mean_squared_error: 0.0044
Epoch 18/20
3/3 [==============================] - 0s 43ms/step - loss: 5.8021 - root_mean_squared_error: 0.0040
Epoch 19/20
3/3 [==============================] - 0s 44ms/step - loss: 5.8032 - root_mean_squared_error: 0.0038
Epoch 20/20
3/3 [==============================] - 0s 43ms/step - loss: 5.7830 - root_mean_squared_error: 0.0040
Epoch 1/20
3/3 [==============================] - 0s 30ms/step - loss: 6.5520 - root_mean_squared_error: 0.0233
Epoch 2/20
3/3 [==============================] - 0s 30ms/step - loss: 7.1608 - root_mean_squared_error: 0.0344
Epoch 3/20
3/3 [==============================] - 0s 32ms/step - loss: 6.2108 - root_mean_squared_error: 0.0153
Epoch 4/20
3/3 [==============================] - 0s 31ms/step - loss: 6.0965 - root_mean_squared_error: 0.0141
Epoch 5/20
3/3 [==============================] - 0s 30ms/step - loss: 5.9757 - root_mean_squared_error: 0.0138


3/3 [==============================] - 0s 38ms/step - loss: 5.6770 - root_mean_squared_error: 0.0078
Epoch 10/20
3/3 [==============================] - 0s 39ms/step - loss: 5.6269 - root_mean_squared_error: 0.0074
Epoch 11/20
3/3 [==============================] - 0s 41ms/step - loss: 5.5869 - root_mean_squared_error: 0.0070
Epoch 12/20
3/3 [==============================] - 0s 42ms/step - loss: 5.5493 - root_mean_squared_error: 0.0068
Epoch 13/20
3/3 [==============================] - 0s 40ms/step - loss: 5.4958 - root_mean_squared_error: 0.0065
Epoch 14/20
3/3 [==============================] - 0s 40ms/step - loss: 5.4637 - root_mean_squared_error: 0.0068
Epoch 15/20
3/3 [==============================] - 0s 39ms/step - loss: 5.4468 - root_mean_squared_error: 0.0068
Epoch 16/20
3/3 [==============================] - 0s 40ms/step - loss: 5.4077 - root_mean_squared_error: 0.0059
Epoch 17/20
3/3 [==============================] - 0s 38ms/step - loss: 5.3837 - root_mean_squared_error: 0.

...

3/3 [==============================] - 0s 37ms/step - loss: 8.9759 - root_mean_squared_error: 0.0282
Epoch 2/20
3/3 [==============================] - 0s 36ms/step - loss: 7.4253 - root_mean_squared_error: 0.0245
Epoch 3/20
3/3 [==============================] - 0s 36ms/step - loss: 6.5683 - root_mean_squared_error: 0.0117
Epoch 4/20
3/3 [==============================] - 0s 35ms/step - loss: 6.4642 - root_mean_squared_error: 0.0086
Epoch 5/20
3/3 [==============================] - 0s 38ms/step - loss: 6.4431 - root_mean_squared_error: 0.0095
Epoch 6/20
3/3 [==============================] - 0s 37ms/step - loss: 6.2966 - root_mean_squared_error: 0.0060
Epoch 7/20
3/3 [==============================] - 0s 40ms/step - loss: 6.2477 - root_mean_squared_error: 0.0061
Epoch 8/20
3/3 [==============================] - 0s 38ms/step - loss: 6.1739 - root_mean_squared_error: 0.0060
Epoch 9/20
3/3 [==============================] - 0s 37ms/step - loss: 6.0544 - root_mean_squared_error: 0.0050
Epo

3/3 [==============================] - 0s 26ms/step - loss: 4.2771 - root_mean_squared_error: 0.0223
Epoch 14/20
3/3 [==============================] - 0s 26ms/step - loss: 4.2262 - root_mean_squared_error: 0.0223
Epoch 15/20
3/3 [==============================] - 0s 25ms/step - loss: 4.1557 - root_mean_squared_error: 0.0214
Epoch 16/20
3/3 [==============================] - 0s 26ms/step - loss: 4.1218 - root_mean_squared_error: 0.0211
Epoch 17/20
3/3 [==============================] - 0s 27ms/step - loss: 4.0990 - root_mean_squared_error: 0.0208
Epoch 18/20
3/3 [==============================] - 0s 28ms/step - loss: 4.0619 - root_mean_squared_error: 0.0207
Epoch 19/20
3/3 [==============================] - 0s 25ms/step - loss: 4.0369 - root_mean_squared_error: 0.0203
Epoch 20/20
3/3 [==============================] - 0s 25ms/step - loss: 4.0288 - root_mean_squared_error: 0.0208
Epoch 1/20
3/3 [==============================] - 0s 43ms/step - loss: 7.0220 - root_mean_squared_error: 0.0

3/3 [==============================] - 0s 40ms/step - loss: 6.6829 - root_mean_squared_error: 0.0087
Epoch 6/20
3/3 [==============================] - 0s 40ms/step - loss: 6.4754 - root_mean_squared_error: 0.0094
Epoch 7/20
3/3 [==============================] - 0s 37ms/step - loss: 6.2869 - root_mean_squared_error: 0.0090
Epoch 8/20
3/3 [==============================] - 0s 35ms/step - loss: 6.1146 - root_mean_squared_error: 0.0087
Epoch 9/20
3/3 [==============================] - 0s 36ms/step - loss: 5.9201 - root_mean_squared_error: 0.0089
Epoch 10/20
3/3 [==============================] - 0s 38ms/step - loss: 5.8136 - root_mean_squared_error: 0.0068
Epoch 11/20
3/3 [==============================] - 0s 35ms/step - loss: 5.7944 - root_mean_squared_error: 0.0078
Epoch 12/20
3/3 [==============================] - 0s 38ms/step - loss: 5.6785 - root_mean_squared_error: 0.0059
Epoch 13/20
3/3 [==============================] - 0s 36ms/step - loss: 5.6238 - root_mean_squared_error: 0.0055

3/3 [==============================] - 0s 27ms/step - loss: 4.1739 - root_mean_squared_error: 0.0196
Epoch 18/20
3/3 [==============================] - 0s 27ms/step - loss: 4.1482 - root_mean_squared_error: 0.0193
Epoch 19/20
3/3 [==============================] - 0s 27ms/step - loss: 4.1253 - root_mean_squared_error: 0.0193
Epoch 20/20
3/3 [==============================] - 0s 27ms/step - loss: 4.0997 - root_mean_squared_error: 0.0191
Epoch 1/20
3/3 [==============================] - 0s 37ms/step - loss: 6.9473 - root_mean_squared_error: 0.0105
Epoch 2/20
3/3 [==============================] - 0s 38ms/step - loss: 8.1934 - root_mean_squared_error: 0.0250
Epoch 3/20
3/3 [==============================] - 0s 38ms/step - loss: 6.8375 - root_mean_squared_error: 0.0124
Epoch 4/20
3/3 [==============================] - 0s 40ms/step - loss: 6.5809 - root_mean_squared_error: 0.0080
Epoch 5/20
3/3 [==============================] - 0s 39ms/step - loss: 6.4881 - root_mean_squared_error: 0.0062


3/3 [==============================] - 0s 35ms/step - loss: 5.7359 - root_mean_squared_error: 0.0084
Epoch 10/20
3/3 [==============================] - 0s 38ms/step - loss: 5.6537 - root_mean_squared_error: 0.0064
Epoch 11/20
3/3 [==============================] - 0s 36ms/step - loss: 5.6249 - root_mean_squared_error: 0.0061
Epoch 12/20
3/3 [==============================] - 0s 34ms/step - loss: 5.5652 - root_mean_squared_error: 0.0049
Epoch 13/20
3/3 [==============================] - 0s 36ms/step - loss: 5.5427 - root_mean_squared_error: 0.0056
Epoch 14/20
3/3 [==============================] - 0s 34ms/step - loss: 5.5523 - root_mean_squared_error: 0.0053
Epoch 15/20
3/3 [==============================] - 0s 36ms/step - loss: 5.5070 - root_mean_squared_error: 0.0048
Epoch 16/20
3/3 [==============================] - 0s 36ms/step - loss: 5.4766 - root_mean_squared_error: 0.0048
Epoch 17/20
3/3 [==============================] - 0s 36ms/step - loss: 5.4669 - root_mean_squared_error: 0.

In [14]:
preds = pd.concat(list_preds)
preds = preds.reset_index()
preds.columns = ['id'] + preds.columns.tolist()[1:]

preds.to_csv(
        os.path.join(EXTERNAL_PATH, "tf_weights_%s.csv" % horizon), index=False
    )

In [15]:
preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.002321,0.002329,0.002314,0.002303,0.002322,0.002309,0.002321,0.002322,0.002330,...,0.002318,0.002305,0.002318,0.002319,0.002326,0.002311,0.002300,0.002320,0.002306,0.002318
1,HOBBIES_1_002_CA_1_validation,0.002582,0.002560,0.002597,0.002631,0.002570,0.002622,0.002587,0.002582,0.002563,...,0.002569,0.002621,0.002585,0.002581,0.002562,0.002598,0.002633,0.002570,0.002623,0.002587
2,HOBBIES_1_003_CA_1_validation,0.002329,0.002304,0.002347,0.002333,0.002328,0.002370,0.002310,0.002328,0.002303,...,0.002327,0.002366,0.002305,0.002325,0.002300,0.002345,0.002331,0.002326,0.002365,0.002303
3,HOBBIES_1_004_CA_1_validation,0.001995,0.001996,0.001972,0.002004,0.001946,0.001997,0.002024,0.001998,0.001998,...,0.001946,0.001997,0.002023,0.001997,0.001996,0.001971,0.002002,0.001945,0.001995,0.002022
4,HOBBIES_1_005_CA_1_validation,0.002299,0.002298,0.002279,0.002297,0.002275,0.002288,0.002306,0.002300,0.002300,...,0.002276,0.002290,0.002308,0.002301,0.002300,0.002280,0.002296,0.002274,0.002286,0.002304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.001246,0.001248,0.001258,0.001255,0.001247,0.001250,0.001257,0.001247,0.001249,...,0.001247,0.001251,0.001258,0.001247,0.001250,0.001259,0.001255,0.001246,0.001250,0.001257
30486,FOODS_3_824_WI_3_validation,0.001327,0.001321,0.001338,0.001326,0.001314,0.001321,0.001332,0.001325,0.001318,...,0.001311,0.001317,0.001328,0.001322,0.001316,0.001333,0.001322,0.001310,0.001317,0.001328
30487,FOODS_3_825_WI_3_validation,0.001179,0.001179,0.001179,0.001180,0.001178,0.001179,0.001177,0.001179,0.001179,...,0.001178,0.001179,0.001177,0.001180,0.001179,0.001179,0.001180,0.001178,0.001180,0.001177
30488,FOODS_3_826_WI_3_validation,0.001166,0.001176,0.001179,0.001175,0.001167,0.001165,0.001179,0.001167,0.001178,...,0.001168,0.001165,0.001180,0.001168,0.001178,0.001180,0.001177,0.001168,0.001165,0.001180
